# Project Topic
To receive full credit for this section, the learner should address the two questions below:
- Is there a clear explanation of what this project is about? 
- Does it give a clear description of generative deep learning models including Generative Adversarial Networks (GANs)?


This project is about generating Lo-fi (low fidelity) music via deep learning. Specifically via LTSM (Long Term, Short Memory) Recurrent Neural Networks. Lo-fi tends to be a genre that has imperfections and raw sounds, often characterized by a relaxed, ambient, and jazzy vibe.

I used this excellent research paper as a reference https://people.idsia.ch/~juergen/blues/IDSIA-07-02.pdf which goes over musical compositions using LTSM RNNs.


# Data 
- Is there a clear description of the data, including the size, dimension, and structure? Some ideas for image data: you can include how many samples, number of channels (color or gray or more?) or modalities, image file format, whether images have the same dimension or not, etc.






The data we'll be using to generate our music are MIDI files. A MIDI file is a type of digital file used for storing music performance data. "MIDI" stands for Musical Instrument Digital Interface. It is a standardized format for communicating between musical devices like keyboards, synthesizers, or computers. Instead of containing actual sound recordings, MIDI files store instructions, like which notes to play, their timing, duration, and dynamics, as well as information about instrument types.

External package requirements:
- ffmpeg
- fluidsynth

In [25]:
import glob
import os

import IPython
import keras
import midi2audio
import pandas as pd
import pretty_midi
import tensorflow as tf
from pydub import AudioSegment

# Using GPU to do training.
print(tf.config.list_physical_devices("GPU"))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [26]:
DATA_DIR = "data/"
EXPORT_DIR = "export/"

In [36]:
# Get all MIDI file paths and check how many there are
midi_paths = glob.glob(f"{DATA_DIR}*.mid")
df_midi = pd.DataFrame(midi_paths, columns=["file_path"])
df_midi.describe()

,file_path
count,93
unique,93
top,data/1.mid
freq,1


Let's analyze how many instruments are in our MIDI files.

In [ ]:
num_instruments = []
for file_path in df_midi["file_path"]:
    midi_file = pretty_midi.PrettyMIDI(file_path)
    num_instruments.append(len(midi_file.instruments))

df_midi["num_instruments"] = num_instruments
df_midi.describe()

/home/basilp/.local/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


,num_instruments
count,93.0
mean,1.0
std,0.0
min,1.0
25%,1.0
50%,1.0
75%,1.0
max,1.0


They're all one, which is fine, we can generate something from this. Let's hear some samples of what the sound like. 

In [60]:
fs = midi2audio.FluidSynth(sound_font="default.sf2", sample_rate=22050)
def convert_midi_to_mp3(midi_file_path, dest_file_path = ""):
    """
    Convert a MIDI file to wav and then to MP3.

    :param midi_file_path: File path of the midi file.
    :param dest_file_path: Path of the destination file. Don't include the file type
    :return: mp3 file path name
    """
    if dest_file_path == "":
        dest_file_path = f"{EXPORT_DIR}mp3/{os.path.splitext(os.path.basename(midi_file_path))[0]}"

    export_file_path_wav = f"{dest_file_path}.wav"
    # Converting MIDI to .wav.
    fs.midi_to_audio(midi_file_path, export_file_path_wav)
    AudioSegment.from_wav(export_file_path_wav).export(f"{dest_file_path}.mp3", format="mp3")
    # Don't need to keep the .wav files anymore after converting to mp3.
    os.remove(export_file_path_wav)
    return f"{dest_file_path}.mp3"

In [61]:
mp3_paths = []
for file_path in df_midi["file_path"]:
    mp3_paths.append(convert_midi_to_mp3(file_path))
df_midi["mp3_paths"] = mp3_paths

FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'export/mp3/1.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'export/mp3/10.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'export/mp3/11.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

Rendering audio to file 'export/mp3/12.wav'..
FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanapp

In [62]:
df_midi["mp3_paths"].head()

0     export/mp3/1.mp3
1    export/mp3/10.mp3
2    export/mp3/11.mp3
3    export/mp3/12.mp3
4    export/mp3/13.mp3
Name: mp3_paths, dtype: object

In [ ]:
IPython.display.Audio(df_midi["mp3_paths"][0])

In [64]:
IPython.display.Audio(df_midi["mp3_paths"][10])

In [65]:
IPython.display.Audio(df_midi["mp3_paths"][40])

In [66]:
IPython.display.Audio(df_midi["mp3_paths"][90])


# Exploratory Data Analysis 
- EDA on images is open-ended, and it can be as simple as picking a few samples of images from each class, displaying them, and describing any findings. Some more ideas: you could plot histograms of those sample images and draw any conclusions. You could summarize image similarity by choosing a single metric to compare. The data may or may not have different image sizes, but you could mention that if the image sizes are different. Also, you can check how the range of the pixel intensities is distributed. You may also count labels (if multi-class or multi-label classification problem) to see if the labels are skewed, or you can plot a histogram of areas by segmentation labels, etc.
10 pts
- EDA goes above and beyond expectations: for example, in addition to picking a few sample images from each class, displaying them, and describing any findings, the learner includes several other histograms, appropriate visualizations, and detailed analysis of the dataset.






# Model Architecture
- Does it describe the model architecture and explain why the learner believes a specific architecture/loss function is suitable for the problem? 
- Did the learner tune the hyperparameters? 
- Did the learner compare multiple architectures/loss functions?
- Note, when you evaluate, don't just base your score on the complexity of the model/technique or performance of the model alone. Base your score on how the learner demonstrates understanding by explaining the concepts and why they chose particular methods/techniques.




# Results and Analysis 
- Does it include results in the form of tables, figures, etc.? 
- Analysis 
- Does it include analysis of results:
- the reasoning of why something worked well or why it didn't work
- description of troubleshooting steps.
- a hyperparameter optimization procedure summary.


# Conclusion 
Does the conclusion include the following:
- Basic reiteration of results
- Discussion of learning and takeaways
- Discussion of why something didn’t work.
- Suggestions for ways to improve.